In [3]:
from torchvision.models import resnet152

In [4]:
model = resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/romilb/.cache/torch/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:10<00:00, 22140447.59it/s]


In [ ]:
Cityscap